In [1]:
# test projection
import os
import dotenv
dotenv.load_dotenv()
import sys
sys.path.append("../")
from map_apis.type_projection import AMapTypeProjectionAdapter


In [2]:
projector = AMapTypeProjectionAdapter()

In [3]:
print(projector.system_prompt)

You map human venue descriptions into AMap type codes based on Taxonomy.

Taxonomy will be in the format of "Type Code • Big Category > Mid Category > Sub Category"
For the type code, the first 2 digits represent big category, middle 2 digits represent mid category, last 2 digits represent sub category.

You should return the type codes that are most related and specific to what user specifies.
Only return codes that exist in the provided taxonomy snippet; never invent new codes.

Known taxonomy:
010000 • Auto Service > Automobile Service Related > Automobile Service Related
010100 • Auto Service > Filling Station > Filling Station
010101 • Auto Service > Filling Station > Sinopec
010102 • Auto Service > Filling Station > PetroChina
010103 • Auto Service > Filling Station > Shell
010104 • Auto Service > Filling Station > Mobil
010105 • Auto Service > Filling Station > Caltex
010107 • Auto Service > Filling Station > FEOSO
010108 • Auto Service > Filling Station > BP PetroChina
010109 •

In [4]:
projector.project_types(["coffee", "school", "universities"])

['141201', '050500', '141200']

Test map tool

In [5]:
from openai import OpenAI
from tools.map import MapTool  # your file

client = OpenAI()

# Initialize your map tool
map_tool = MapTool()

In [6]:
# prompt
system_prompt = "You are a location reasoning assistant that can query map data."

# Build the response
response = client.responses.create(
    model="gpt-4.1",  # or gpt-4o-mini, etc.
    instructions = system_prompt,
    input= "Find coffee shops near Shanghai Tower.",
    tools=map_tool.tools,  # Use the JSON schemas
)


In [7]:
import json
tool = response.output[0].name
arguments = json.loads(response.output[0].arguments)
map_tool.call(tool, arguments)

{'provider': 'amap',
 'place_id': '310000',
 'name': '上海市',
 'category': 'unknown',
 'subcategory': None,
 'lat': 31.230525,
 'lng': 121.473667,
 'raw': {'formatted_address': '上海市',
  'country': '中国',
  'province': '上海市',
  'citycode': '021',
  'city': '上海市',
  'district': [],
  'township': [],
  'neighborhood': {'name': [], 'type': []},
  'building': {'name': [], 'type': []},
  'adcode': '310000',
  'street': [],
  'number': [],
  'location': '121.473667,31.230525',
  'level': '省'}}

call tools 1 by1

In [22]:
map_tool.call("getMapVisualization", {"origin": {"lat": 31.230525, "lng": 121.473667}})

{'provider': 'amap',
 'url': 'https://restapi.amap.com/v3/staticmap?location=121.473667%2C31.230525&zoom=13&size=750%2A520&markers=mid%2C%2CA%3A121.473667%2C31.230525&key=68913ce8db83f7bbe5b4cca9720c7a8f',
 'query_params': {'location': '121.473667,31.230525',
  'zoom': 13,
  'size': '750*520',
  'markers': 'mid,,A:121.473667,31.230525'},
 'origin': {'lat': 31.230525, 'lng': 121.473667},
 'overlays': []}

In [9]:
map_tool.call("getNearbyPlaces", {"origin": {"lat": 31.230525, "lng": 121.473667}, "descriptive_types": ["university", "coffee shops"], "radius":500, "rank":"DISTANCE"})

[{'provider': 'amap',
  'place_id': 'B0K60H1UF4',
  'name': 'Aisle No.300-Drama Cafe',
  'category': '050500',
  'subcategory': None,
  'lat': 31.229652,
  'lng': 121.471218,
  'rating': None,
  'review_count': None,
  'address': '上海大剧院C1入口左侧',
  'raw': {'parent': '',
   'address': '上海大剧院C1入口左侧',
   'distance': '252',
   'pcode': '310000',
   'adcode': '310101',
   'pname': '上海市',
   'cityname': '上海市',
   'type': '餐饮服务;咖啡厅;咖啡厅',
   'typecode': '050500',
   'adname': '黄浦区',
   'citycode': '021',
   'name': 'Aisle No.300-Drama Cafe',
   'location': '121.471218,31.229652',
   'id': 'B0K60H1UF4'}},
 {'provider': 'amap',
  'place_id': 'B0FFGYUGT8',
  'name': '星巴克(迪美广场店)',
  'category': '050501',
  'subcategory': None,
  'lat': 31.228141,
  'lng': 121.473389,
  'rating': None,
  'review_count': None,
  'address': '人民大道221号B1层236-238号单元',
  'raw': {'parent': 'B00155IVIY',
   'address': '人民大道221号B1层236-238号单元',
   'distance': '266',
   'pcode': '310000',
   'adcode': '310101',
   'pname': '上海市

In [5]:
map_tool.tools

[{'type': 'function',
  'name': 'getPlaceInfo',
  'description': 'Resolve an address or place description to a normalized map feature using the amap API.',
  'parameters': {'type': 'object',
   'properties': {'address': {'type': 'string',
     'description': 'Full address or POI description to geocode.'},
    'language': {'type': 'string',
     'description': 'Optional language hint understood by the provider.'},
    'extra_params': {'type': 'object',
     'description': 'Advanced provider-specific parameters.',
     'additionalProperties': True}},
   'required': ['address']}},
 {'type': 'function',
  'name': 'getMapVisualization',
  'description': 'Produce a static visualization URL that highlights the origin and optional overlay markers.',
  'parameters': {'type': 'object',
   'properties': {'origin': {'type': 'object',
     'description': 'Origin to highlight.',
     'properties': {'lat': {'type': 'number',
       'description': 'Latitude in decimal degrees.'},
      'lng': {'type':

In [5]:
map_tool.call("getDistances", {"origin": {"lat": 31.230525, "lng": 121.473667}, "destinations": [{"lat": 31.12, "lng": 121.48}], "mode": "bus"})

[{'origin_id': '1', 'dest_id': '1', 'distance': '16339', 'duration': '1346'}]


[{'origin': {'lat': 31.230525, 'lng': 121.473667},
  'destination': {'lat': 31.12, 'lng': 121.48},
  'distance_m': 16339.0,
  'duration_s': 1346.0,
  'mode': 'bus',
  'raw': {'origin_id': '1',
   'dest_id': '1',
   'distance': '16339',
   'duration': '1346'}}]

# test Railtrack

In [1]:
import sys
sys.path.append('../')
from tools.map_rt import *

In [2]:
import railtracks as rt
get_place_info_tool = rt.function_node(getPlaceInfo)

In [3]:
GeoAgent = rt.agent_node(
                name="GeoAgent",
                tool_nodes=[
                  get_place_info_tool, 
                ],    # the agent has access to these tools
                llm = rt.llm.OpenAILLM("gpt-4o"),
            )

# run the agent
result = await rt.call(GeoAgent, "Where is the Shanghai Tower?")

[+7.287  s] RT.Validation: WARNING  - No SystemMessage was provided. This is not recommended. Please provide the first message as a SystemMessage.
[+7.289  s] RT.Validation: WARNING  - You have set max_tool_calls to None. The llm will be able to make unlimited tool calls.
[+7.290  s] RT          : INFO     - START CREATED GeoAgent
[+9.662  s] RT          : INFO     - GeoAgent CREATED getPlaceInfo
[+11.024 s] RT          : INFO     - getPlaceInfo DONE
[+13.880 s] RT          : INFO     - GeoAgent DONE
[+13.884 s] RT.Session  : INFO     - Saving execution info to .railtracks/427a30bb-bd53-4286-b408-9e3655f8640d.json


In [4]:
result

LLMResponse(The Shanghai Tower is located in Shanghai, China. The coordinates for its approximate location are latitude 31.230525 and longitude 121.473667.)